In [1]:
import requests
import datetime as dt
import time
import json
import threading
from sqlalchemy.dialects.postgresql import insert
import pandas as pd

In [2]:
def get_tmc_data_by_url(url):
    headers = { 
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0"
        }

    response = requests.get(url, headers= headers)
    result = response.json()

    return result


#################### tsetmc api
def get_index_historic_data(index_type_code):
    
    url = f'https://cdn.tsetmc.com/api/Index/GetIndexB2History/{index_type_code}'
    raw_data = get_tmc_data_by_url(url)
    
    return raw_data

def get_fund_list_data():

    fund_list_file = 'data/fund_metadata.json'
    with open(fund_list_file, 'r') as file:
        raw_data = json.load(file)
    
    return raw_data

def get_fund_historic_data(fund_id):

    url = f'https://cdn.tsetmc.com/api/ClosingPrice/GetClosingPriceDailyList/{fund_id}/0'
    raw_data = get_tmc_data_by_url(url)
    
    return raw_data


#################### transform tsetmc data
def transform_index_historic_data(raw_data, index_type, index_code):
    
    # change to dataframe
    df = pd.DataFrame(raw_data['indexB2'])

    # get interest data
    df['Index_Id'] = df['insCode']
    df['Index_Type'] = index_type
    df['Date_En'] = df['dEven'].astype(str).apply(lambda x: dt.datetime(int(x[:4]), int(x[4:6]), int(x[6:8])))
    df['Value'] = df['xNivInuPbMresIbs']
    df['Previous_Value'] = df['xNivInuClMresIbs']
    df['Closed_Day'] = False # available days are not closed days
    
    # remove extra data
    df = df[['Index_Id', 'Date_En', 'Index_Type', 'Value', 'Previous_Value', 'Closed_Day']]

    # build time frame and merge it with dataframe
    time_frame = pd.DataFrame({
        'Date_En': pd.date_range(start= '2022-01-01', end= dt.datetime.now()),
        'Index_Id': index_code,
        'Index_Type': index_type
        })
    df = pd.merge(time_frame, df, how= 'left')

    # mark new days as closed days
    df['Closed_Day'] = df['Closed_Day'].fillna(True)
    # backfill null values
    df[['Value', 'Previous_Value']] = df[['Value', 'Previous_Value']].ffill()

    return df

def transform_fund_list_data(raw_data):
    
    # change to dataframe
    df = pd.DataFrame(raw_data)

    # get interest data
    df['Fund_Id'] = df['insCode']
    df['Fund_Name'] = df['lVal18AFC']
    df['Fund_NameDetail'] = df['name']
    df['Fund_TypeNumber'] = df['typeNum']
    df['Fund_TypeName'] = df['typeName']

    # remove extra data
    df = df[['Fund_Id', 'Fund_Name', 'Fund_NameDetail', 'Fund_TypeNumber', 'Fund_TypeName']]

    return df

def transform_fund_historic_data(raw_data, fund_name, fund_name_detail, fund_type_number, fund_type_name):

    # change to dataframe
    df = pd.DataFrame(raw_data['closingPriceDaily'])

    # get interest data
    df['Fund_Id'] = df['insCode']
    df['Fund_Name'] = fund_name
    df['Fund_NameDetail'] = fund_name_detail
    df['Fund_TypeNumber'] = fund_type_number
    df['Fund_TypeName'] = fund_type_name
    df['Date_En'] = df['dEven'].astype(str).apply(lambda x: dt.datetime(int(x[:4]), int(x[4:6]), int(x[6:8])))
    df['Price_Closing'] = df['pClosing']
    df['Price_Yesterday'] = df['priceYesterday']
    df['Closed_Day'] = False

    # remove extra data
    df = df[['Fund_Id', 'Date_En', 'Fund_Name', 'Fund_NameDetail', 'Fund_TypeNumber', 'Fund_TypeName', 'Price_Closing', 'Price_Yesterday', 'Closed_Day']]

    # build time frame and merge it with dataframe
    time_frame = pd.DataFrame({
        'Date_En': pd.date_range(start='2022-01-01', end=dt.datetime.now()),
        'Fund_Id': df['Fund_Id'].iloc[0]
        })
    df = pd.merge(time_frame, df, how= 'left')

    # mark new days as closed days
    df['Closed_Day'] = df['Closed_Day'].fillna(True)
    # backfill null values
    df['Fund_Name'] = df['Fund_Name'].fillna(fund_name)
    df['Fund_NameDetail'] = df['Fund_NameDetail'].fillna(fund_name_detail)
    df['Fund_TypeNumber'] = df['Fund_Name'].fillna(fund_type_number)
    df['Fund_TypeName'] = df['Fund_NameDetail'].fillna(fund_type_name)
    df[['Price_Closing', 'Price_Yesterday']] = df[['Price_Closing', 'Price_Yesterday']].ffill()

    return df



In [3]:
raw_fund_metadata = get_fund_list_data()
fund_metadata = transform_fund_list_data(raw_fund_metadata)

In [4]:
fund_metadata

,Fund_Id,Fund_Name,Fund_NameDetail,Fund_TypeNumber,Fund_TypeName
0,17914401175772326,اهرم,سهامی اهرمی کاریزما,622,صندوق اهرمی
1,67675656072510693,پالايش,پالایشی یکم,623,صندوق بخشی | پتروپالایش
2,62235397452612911,دارا يكم,واسطه گری مالی یکم,623,صندوق بخشی | بانک و بیمه
3,64216772923447100,شتاب,سهامی اهرمی شتاب آگاه,622,صندوق اهرمی
4,41927452991671109,توان,سهامی اهرمی توان مفید,622,صندوق اهرمی
...,...,...,...,...,...
182,30582275818828857,ناب,در اوراق بهادار مبتنی بر سکه طلا نهایت نگر,32,صندوق مبتنی بر طلا
183,28374437855144739,آلتون,طلای آسمان آلتون,32,صندوق مبتنی بر طلا
184,9089296888187061,تابش,طلای تابان تمدن,32,صندوق مبتنی بر طلا
185,34144395039913458,عیار,طلای عیار مفید,32,صندوق مبتنی بر طلا


In [6]:
for row in fund_metadata.iloc[:1].itertuples():
    raw_data = get_fund_historic_data(row.Fund_Id)
    df = transform_fund_historic_data(raw_data, row.Fund_Name, row.Fund_NameDetail, row.Fund_TypeNumber, row.Fund_TypeName)
    # insert_fund_historic_data(df)

In [7]:
df

,Date_En,Fund_Id,Fund_Name,Fund_NameDetail,Fund_TypeNumber,Fund_TypeName,Price_Closing,Price_Yesterday,Closed_Day
0,2022-01-01,17914401175772326,اهرم,سهامی اهرمی کاریزما,اهرم,سهامی اهرمی کاریزما,10006.0,10000.0,False
1,2022-01-02,17914401175772326,اهرم,سهامی اهرمی کاریزما,اهرم,سهامی اهرمی کاریزما,10022.0,10006.0,False
2,2022-01-03,17914401175772326,اهرم,سهامی اهرمی کاریزما,اهرم,سهامی اهرمی کاریزما,10004.0,10022.0,False
3,2022-01-04,17914401175772326,اهرم,سهامی اهرمی کاریزما,اهرم,سهامی اهرمی کاریزما,9996.0,10004.0,False
4,2022-01-05,17914401175772326,اهرم,سهامی اهرمی کاریزما,اهرم,سهامی اهرمی کاریزما,9981.0,9996.0,False
...,...,...,...,...,...,...,...,...,...
870,2024-05-20,17914401175772326,اهرم,سهامی اهرمی کاریزما,اهرم,سهامی اهرمی کاریزما,20270.0,20270.0,False
871,2024-05-21,17914401175772326,اهرم,سهامی اهرمی کاریزما,اهرم,سهامی اهرمی کاریزما,20460.0,20270.0,False
872,2024-05-22,17914401175772326,اهرم,سهامی اهرمی کاریزما,اهرم,سهامی اهرمی کاریزما,20460.0,20270.0,True
873,2024-05-23,17914401175772326,اهرم,سهامی اهرمی کاریزما,اهرم,سهامی اهرمی کاریزما,20460.0,20270.0,True
